In [60]:
from mpi4py import MPI
from petsc4py import PETSc

from tqdm.notebook import tqdm_notebook

import numpy as np

import ufl

from dolfinx import fem, io, mesh, plot, la
from dolfinx.fem.petsc import assemble_matrix, create_vector, apply_lifting
from basix.ufl import element
from dolfinx.io import XDMFFile

import pyvista

https://www.wias-berlin.de/people/john/BETREUUNG/master_ding.pdf
https://www.youtube.com/watch?v=LflB3-bEK2M
http://redbkit.github.io/redbKIT/math/AdvDiffReact/#id

In [61]:
# create mesh for calculations to take place in
msh = mesh.create_unit_square(MPI.COMM_WORLD, 16,16)

# define elements to be used for the velocity and the distribution
z_p1 = element("P", msh.basix_cell(), 1)
v_p2 = element("P", msh.basix_cell(), 2, shape=(msh.geometry.dim,))

# Define function spaces to be used for the two variables
Z = fem.FunctionSpace(msh, z_p1)
V = fem.FunctionSpace(msh, v_p2)

In [62]:
# define functions to be used for boundaries and initials
def initial_condition(x, a=5):
    return np.exp(-a * (x[0]**2 + x[1]**2))

def flow(x):
    return np.stack((np.ones(x.shape[1]), np.zeros(x.shape[1]))) *20
    
# Function to mark the lid (y = 1)
def lid(x):
    return np.isclose(x[1], 1)

def inflow_point(x):
    return np.isclose(x[0], 0) & np.isclose(x[1], 0.5)

# define time step variables 
T = 1.3          # final time
num_steps = 1300   # number of time steps
dt = T / num_steps # time step size

In [63]:
# define boundary conditions
inflow_dofs = fem.locate_dofs_geometrical(Z, lid)
bc_inflow = fem.dirichletbc(PETSc.ScalarType(1), inflow_dofs, Z)
bc_outlfow = fem.dirichletbc(PETSc.ScalarType(-1), inflow_dofs, Z)

# order here is important as want to apply outflow before the inflow
bcs = [bc_outlfow, bc_inflow]
# bcs = []

In [64]:
# Create variables for results at current and previous timesteps
s_ = fem.Function(Z)
# s_.interpolate(initial_condition)
s_n = fem.Function(Z)
# s_n.interpolate(initial_condition)

# very basic flow variable
u_n = fem.Function(V)
u_n.interpolate(flow)

# create trial and test functions
s = ufl.TrialFunction(Z)
z = ufl.TestFunction(Z)

In [65]:
# https://github.com/unifem/fenics-notes/blob/master/notebooks/advection-diffusion-reaction.ipynb
# https://jsdokken.com/dolfinx-tutorial/chapter2/diffusion_code.html

# constants for calculations
rho = fem.Constant(msh, PETSc.ScalarType(10)) # advection coefficent
eps = fem.Constant(msh, PETSc.ScalarType(1)) # diffusion coefficent
k = fem.Constant(msh, dt)
fs = fem.Constant(msh, PETSc.ScalarType(0))

# https://fenicsproject.discourse.group/t/navier-stokes-and-convection-diffusion/7170
F = rho*((s - s_n) / k) *z*ufl.dx \
   + rho*ufl.dot(u_n, ufl.grad(s_n))*z*ufl.dx \
   + eps * ufl.dot(ufl.grad(s), ufl.grad(z))* ufl.dx \
   - fs*z*ufl.dx

a = fem.form(ufl.lhs(F))
L = fem.form(ufl.rhs(F))

# Just diffusion equation
# https://jsdokken.com/dolfinx-tutorial/chapter2/diffusion_code.html
# a = fem.form(s * z * ufl.dx + dt * ufl.dot(ufl.grad(s), ufl.grad(z)) * ufl.dx)
# L = fem.form((s_n + dt * fs) * z * ufl.dx)

# create parts of the equation that will not change between timesteps
# https://fenicsproject.discourse.group/t/how-to-set-bcs-for-petsc-matrices-and-petsc-vectors-in-dolfinx/4135
A = assemble_matrix(a, bcs=bcs)
A.assemble()
b = create_vector(L)

# create solver for the linear (matrix) problem
## Using petsc4py to create a linear solver
# As we have already assembled `a` into tbhe matrix `A`, we can no longer use the `dolfinx.fem.petsc.LinearProblem` class to solve the problem. Therefore, we create a linear algebra solver using PETSc, and assign the matrix `A` to the solver, and choose the solution strategy.
solver2 = PETSc.KSP().create(msh.comm)
solver2.setOperators(A)
solver2.setType(PETSc.KSP.Type.PREONLY)
solver2.getPC().setType(PETSc.PC.Type.LU)

In [66]:
# https://jsdokken.com/dolfinx-tutorial/chapter2/diffusion_code.html
# create output file
xdmf = io.XDMFFile(msh.comm, "results/diffusion.xdmf", "w")
xdmf.write_mesh(msh)

In [67]:
# inital time
t = 0
xdmf.write_function(s_,t)

for i in tqdm_notebook(range(num_steps)):
    t += dt # update time

    # https://jsdokken.com/dolfinx-tutorial/chapter2/diffusion_code.html
    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    fem.petsc.assemble_vector(b, L)
    apply_lifting(b, [a], [bcs])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
    fem.set_bc(b, bcs)

    # solve
    solver2.solve(b, s_.vector)
    s_.x.scatter_forward() # correct for inacuracies 

    s_n.x.array[:] = s_.x.array # update previous step

    xdmf.write_function(s_,t)

xdmf.close()

  0%|          | 0/1300 [00:00<?, ?it/s]

In [68]:
# visualisation
pyvista.set_jupyter_backend('client')

pyvista.start_xvfb()


topologyZ, cell_typesZ, geometryZ = plot.vtk_mesh(Z)
grid0 = pyvista.UnstructuredGrid(topologyZ, cell_typesZ, geometryZ)
grid0.point_data["s0"] = s_n.x.array

grid1 = pyvista.UnstructuredGrid(topologyZ, cell_typesZ, geometryZ)
grid1.point_data["s1"] = s_.x.array

topologyV, cell_typesV, geometryV = plot.vtk_mesh(V)
values = np.zeros((geometryV.shape[0], 3), dtype=np.float64)
values[:, :len(u_n)] = u_n.x.array.real.reshape((geometryV.shape[0], len(u_n)))
velocity_grid = pyvista.UnstructuredGrid(topologyV, cell_typesV, geometryV)
velocity_grid["u"] = values
# Create a point cloud of glyphs
glyphs = velocity_grid.glyph(orient="u", factor=0.2)

plotter = pyvista.Plotter(notebook=True, shape=(1, 2))
plotter.subplot(0,0)
plotter.add_mesh(grid0, show_edges=True)
plotter.view_xy()
plotter.subplot (0,1)
plotter.add_mesh(grid1, show_edges=True)
plotter.view_xy()
#plotter.add_mesh(glyphs)

plotter.show()

Widget(value='<iframe src="http://localhost:36879/index.html?ui=P_0x7fa49da59540_3&reconnect=auto" class="pyvi…

In [28]:
# f = fem.Constant(msh, PETSc.ScalarType(0))
# a = u * v * ufl.dx + dt * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
# L = (u_n + dt * f) * v * ufl.dx